In [20]:
import tensorflow as tf
import numpy as np

In [21]:
tf.config.experimental_run_functions_eagerly(True)

In [22]:
def generate_test_input(N):
    # Gerar boxes preditas e alvo aleatórias
    boxes_pred = np.random.rand(N, 5)*10  # N amostras de boxes preditas
    target_boxes = np.random.rand(N, 5)*10  # N amostras de boxes alvo

    # Adicionar valores específicos para o ângulo em radianos
    angle_values = np.random.uniform(low=0, high=2*np.pi, size=(N, 1))  # N valores de ângulo em radianos
    boxes_pred[:, 4] = angle_values.flatten()  # Adicionar ângulos às boxes preditas
    target_boxes[:, 4] = angle_values.flatten()  # Adicionar ângulos às boxes alvo
    
    return boxes_pred, target_boxes

In [23]:
boxes_predd, target_boxess = generate_test_input(1)

In [24]:
def probiou_loss_og(boxes_pred, target_boxes_, EPS = 1e-3, mode='l2'):

    """
        boxes_pred    -> a matrix [N,5](x,y,w,h,angle - in radians) containing ours predicted box ;in case of HBB angle == 0
        target_boxes_  -> a matrix [N,5](x,y,w,h,angle - in radians) containing ours target    box ;in case of HBB angle == 0
        EPS     -> threshold to avoid infinite values
        mode       -> ('l1' in [0,1] or 'l2' in [0,inf]) metrics according our paper

    """

    x1, y1, w1, h1, theta1 = tf.unstack(boxes_pred, axis=1)
    x2, y2, w2, h2, theta2 = tf.unstack(target_boxes_, axis=1)
    x1 = tf.reshape(x1, [-1, 1])
    y1 = tf.reshape(y1, [-1, 1])
    h1 = tf.reshape(h1, [-1, 1])
    w1 = tf.reshape(w1, [-1, 1])
    theta1 = tf.reshape(theta1, [-1, 1])
    x2 = tf.reshape(x2, [-1, 1])
    y2 = tf.reshape(y2, [-1, 1])
    h2 = tf.reshape(h2, [-1, 1])
    w2 = tf.reshape(w2, [-1, 1])
    theta2 = tf.reshape(theta2, [-1, 1])

    # gbb form
    aa = w1**2/12; bb = h1**2/12; angles = theta1
    # rotated form
    a1 = aa*tf.math.pow(tf.math.cos(angles), 2.) + bb*tf.math.pow(tf.math.sin(angles), 2.)
    b1 = aa*tf.math.pow(tf.math.sin(angles), 2.) + bb*tf.math.pow(tf.math.cos(angles), 2.)
    c1 = 0.5*(aa - bb)*tf.math.sin(2.*angles)

    # gbb form
    aa = w2**2/12; bb = h2**2/12; angles = theta2
    # rotated form
    a2 = aa*tf.math.pow(tf.math.cos(angles), 2.) + bb*tf.math.pow(tf.math.sin(angles), 2.)
    b2 = aa*tf.math.pow(tf.math.sin(angles), 2.) + bb*tf.math.pow(tf.math.cos(angles), 2.)
    c2 = 0.5*(aa - bb)*tf.math.sin(2.*angles)

    B1 = 1/4.*( (a1+a2)*(y1-y2)**2. + (b1+b2)*(x1-x2)**2. ) + 1/2.*( (c1+c2)*(x2-x1)*(y1-y2) )
    # B1 = B1 / ( (a1+a2)*(b1+b2) - (c1+c2)**2. + EPS )
    B1 = B1 / ( (a1+a2)*(b1+b2) - (c1+c2)**2.)

    print("valor de b1 na loss OG:{}".format(B1))
    sqrt = (a1*b1-c1**2)*(a2*b2-c2**2)
    # sqrt = tf.clip_by_value(sqrt, EPS, tf.reduce_max(sqrt)+EPS)
    # sqrt = tf.clip_by_value(sqrt, EPS, tf.reduce_max(sqrt))
    # B2 = ( (a1+a2)*(b1+b2) - (c1+c2)**2. )/( 4.*tf.math.sqrt(sqrt) + EPS )
    B2 = ( (a1+a2)*(b1+b2) - (c1+c2)**2. )/( 4.*tf.math.sqrt(sqrt))
    #B2 = tf.clip_by_value(B2, EPS, tf.reduce_max(B2)+EPS)
    B2 = 1/2.*tf.math.log(B2)

    # print("valor de b2 na LOSS OG{}".format(B2))
    
    Bd = B1 + B2
    Bd = tf.clip_by_value(Bd, EPS, 100.)

    l1 = tf.math.sqrt(1 - tf.math.exp(-Bd) + EPS)

    if mode=='l2':
        l2 = tf.math.pow(l1, 2.)
        probiou = - tf.math.log(1. - l2 + EPS)
    else:
        probiou = l1

    return B1

In [45]:
def probiou_lossTF(boxes_pred, target_boxes_, EPS = 1e-3, mode='l2'):

    """
        boxes_pred    -> a matrix [N,5](x,y,w,h,angle - in radians) containing ours predicted box ;in case of HBB angle == 0
        target_boxes_  -> a matrix [N,5](x,y,w,h,angle - in radians) containing ours target    box ;in case of HBB angle == 0
        EPS     -> threshold to avoid infinite values
        mode       -> ('l1' in [0,1] or 'l2' in [0,inf]) metrics according our paper

    """

    x1, y1, w1, h1, theta1 = tf.unstack(boxes_pred, axis=1)
    x2, y2, w2, h2, theta2 = tf.unstack(target_boxes_, axis=1)
    x1 = tf.reshape(x1, [-1])
    y1 = tf.reshape(y1, [-1])
    h1 = tf.reshape(h1, [-1])
    w1 = tf.reshape(w1, [-1])
    theta1 = tf.reshape(theta1, [-1])
    x2 = tf.reshape(x1, [-1])
    y2 = tf.reshape(y2, [-1])
    h2 = tf.reshape(h2, [-1])
    w2 = tf.reshape(w2, [-1])
    theta2 = tf.reshape(theta2, [-1])

    # gbb form
    # print("O valor de w1 eh: {}".format(w1))
    # print("O valor de h1 eh: {}".format(h1))
    # print("O valor de theta1 eh: {}".format(theta1))
    aa = w1**2/12; bb = h1**2/12; angles = theta1
    # rotated form
    a1 = aa*tf.math.pow(tf.math.cos(angles), 2.) + bb*tf.math.pow(tf.math.sin(angles), 2.)
    b1 = aa*tf.math.pow(tf.math.sin(angles), 2.) + bb*tf.math.pow(tf.math.cos(angles), 2.)
    c1 = 0.5*(aa - bb)*tf.math.sin(2.*angles)

    # gbb form
    aa = w2**2/12; bb = h2**2/12; angles = theta2
    # rotated form
    a2 = aa*tf.math.pow(tf.math.cos(angles), 2.) + bb*tf.math.pow(tf.math.sin(angles), 2.)
    b2 = aa*tf.math.pow(tf.math.sin(angles), 2.) + bb*tf.math.pow(tf.math.cos(angles), 2.)
    c2 = 0.5*(aa - bb)*tf.math.sin(2.*angles)
    
    # print(tf.rank(a1))
    # print("a1:{}".format(a1))
    # print("c1:{}".format(c1))
    # print("b1:{}".format(b1))

    # Formulação matricial
    sigma1 = tf.Variable([[a1, c1], [c1, b1]])
    
    sigma2 = tf.Variable([[a2, c2], [c2, b2]])
    
    sigma = (1/2)*(sigma1 + sigma2)
    
    mu1 = tf.Variable([x1, y1])
    
    mu2 = tf.Variable([x2, y2])
    
    
    print(sigma.shape)
    teste = tf.linalg.inv(sigma)
    
    pt1 = (1/8)*tf.transpose((mu1-mu2))*tf.linalg.inv(sigma)
    
    print(pt1)
    
    Bd = b1+b2
    
    Bd = tf.clip_by_value(Bd, EPS, 100.)
    
    print(Bd.shape)

    l1 = tf.math.sqrt(1 - tf.math.exp(-Bd) + EPS)
    
    if mode=='l2':
        l2 = tf.math.pow(l1, 2.)
        probiou = - tf.math.log(1. - l2 + EPS)
    else:
        probiou = l1
        
    return b1


In [46]:
digs3 = probiou_lossTF(boxes_predd, target_boxess)

(2, 2, 1)


InvalidArgumentError: {{function_node __wrapped__MatrixInverse_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input matrix must be square. [Op:MatrixInverse] name: 

In [7]:
digs1 = probiou_loss(boxes_predd, target_boxess)

(5, 1, 1)
[[[0.23230047]]

 [[1.62623414]]

 [[0.19013321]]

 [[0.02992921]]

 [[0.12843231]]]
(5, 1, 5)


In [8]:
digs2 = probiou_loss_og(boxes_predd, target_boxess)

valor de b1 na loss OG:[[0.34086725]
 [1.48973503]
 [0.37616205]
 [0.5539507 ]
 [7.46741721]]


In [9]:
diferenca = digs1 - digs2
print(diferenca)

tf.Tensor(
[[[-0.10856678]
  [-1.25743456]
  [-0.14386158]
  [-0.32165023]
  [-7.23511674]]

 [[ 1.28536689]
  [ 0.13649911]
  [ 1.25007209]
  [ 1.07228344]
  [-5.84118307]]

 [[-0.15073404]
  [-1.29960182]
  [-0.18602884]
  [-0.36381749]
  [-7.277284  ]]

 [[-0.31093804]
  [-1.45980582]
  [-0.34623284]
  [-0.52402149]
  [-7.43748799]]

 [[-0.21243494]
  [-1.36130272]
  [-0.24772974]
  [-0.42551839]
  [-7.3389849 ]]], shape=(5, 5, 1), dtype=float64)


In [ ]:
w = 0.7301464
h1 = 0.5703776
theta1 = 4.0121395

aa = w**2/12; bb = h1**2/12; angles = theta1
    # rotated form
a1 = aa*tf.math.pow(tf.math.cos(angles), 2.) + bb*tf.math.pow(tf.math.sin(angles), 2.)
b1 = aa*tf.math.pow(tf.math.sin(angles), 2.) + bb*tf.math.pow(tf.math.cos(angles), 2.)
c1 = 0.5*(aa - bb)*tf.math.sin(2.*angles)

# Criando a matriz sigma
sigma = tf.stack([[a1, c1],
                  [c1, b1]])